# Leitura e filtro das bases

In [ ]:
PATH_TO_LOAD_FILES = r'/content/drive/MyDrive/chess_dataset/csv_files' # use your path
PATH_TO_SAVE_FILES = r'/content/drive/MyDrive/chess_dataset/chess_files' # use your path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
import glob

path = PATH_FILES 
all_files_info = glob.glob(path + "/info_*.csv")
all_files_pgn = glob.glob(path + "/pgn_*.csv")

In [ ]:
df_info = pd.concat((pd.read_csv(f) for f in all_files_info))

print(df_info.shape)

df_info.head()

(3660164, 6)


,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result
0,666000,2380.0,NaN,81,tourn,1/2-1/2
1,666001,2350.0,NaN,80,tourn,1-0
2,666002,2355.0,NaN,89,tourn,1/2-1/2
3,666003,NaN,2365.0,30,tourn,1/2-1/2
4,666004,NaN,2370.0,72,tourn,1-0


In [ ]:
df_pgn = pd.concat((pd.read_csv(f) for f in all_files_pgn))

print(df_pgn.shape)

df_pgn.head()

(3660164, 2)


,id_partida,pgn
0,666000,b' 1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. ...
1,666001,b' 1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. N...
2,666002,b' 1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. ...
3,666003,b' 1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. B...
4,666004,b' 1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. ...


## Filtro dos jogos relevantes

In [ ]:
game_types = ['tourn', 'match', 'game', 'swiss']

df_info = df_info[df_info['EventType'].isin(game_types)]

print(df_info.shape)

(2523852, 6)


In [ ]:
valid_results = ['1-0', '0-1', '1/2-1/2']

df_info = df_info[df_info['Result'].isin(valid_results)]

print(df_info.shape)

(2522056, 6)


In [ ]:
df_games = df_info.merge(df_pgn, how='left')

df_games = df_games[~df_games['pgn'].isna()]

df_games = df_games[~(df_games['pgn'].apply(lambda x: len(x)) == 8)]

df_games.head()

,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result,pgn
0,666000,2380.0,NaN,81,tourn,1/2-1/2,b' 1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. ...
1,666001,2350.0,NaN,80,tourn,1-0,b' 1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. N...
2,666002,2355.0,NaN,89,tourn,1/2-1/2,b' 1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. ...
3,666003,NaN,2365.0,30,tourn,1/2-1/2,b' 1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. B...
4,666004,NaN,2370.0,72,tourn,1-0,b' 1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. ...


In [ ]:
df_games.loc[df_games.sample(1).index, 'pgn'].values[0]

"b' 1. d4 d5 2. c4 e6 3. Nf3 c5 4. cxd5 exd5 5. g3 Nc6 6. Bg2 Nf6 7. O-O Be7 8. dxc5 Bxc5 9. a3 O-O 10. b4 Bb6 11. Bb2 Ne4 12. Nc3 Be6 13. Na4 Bc7 14. Nd4 Qe7 15. Rc1 Nxd4 16. Qxd4 f6 17. Bxe4 dxe4 18. Qxe4 Bb6 19. Nxb6 axb6 20. Bd4 Qd6 21. Rfd1 Bd5 22. Qd3 Rfd8 23. e4 Be6 24. Qf3 Rac8 25. Rxc8 Bxc8 26. Rd2 Be6 27. Be3 Qe7 28. Rxd8+ Qxd8 29. Qe2 Kf7 30. Qh5+ Kf8 31. Kg2 h6 32. Qb5 Qc8 33. f3 Bh3+ 34. Kf2 Qc2+ 35. Qe2 Qb3 36. Bxb6 Qxa3 37. Bc5+ Kg8 38. g4 h5 39. gxh5 Qc1 40. Kg3 Bf1 41. Qe3 Qc2 42. Qg1 Bc4 43. h6 Qd2 44. Qe3 Qd1 45. hxg7 Kxg7 46. Bd4 Kg6 47. Bxf6 Kxf6 48. Qc3+ Kg5 49. h4+ Kh5 50. Qxc4 Qg1+ 51. Kf4 Kxh4 52. e5 Qg5+ 53. Ke4 Kg3 54. Qf7 Qd2 55. Qg7+ Kf2 56. Qxb7 Qc2+ 57. Kd4 Qb2+ 58. Kd5 Kxf3 59. e6 Kf4 60. Qb8+ Kf5 61. Qf8+ Kg5 62. Qc5 Kg6 63. b5 Qb3+ 64. Kd6 Qg3+ 65. Kd7 Qd3+ 66. Ke8 Qe4 67. Qc6 Qe5 68. e7+ Kg7 69. b6 Qh5+ 70. Kd8 1-0'"

## Filtro de comentários e variantes

In [ ]:
import re

In [ ]:
df_games['pgn'] = df_games['pgn'].apply(lambda g: re.sub(r'\([^)]*\)', '', g)) # remove sidelines

df_games['pgn'] = df_games['pgn'].apply(lambda g: re.sub(r'\{[^)]*\}', '', g)) # remove comentários

df_games['pgn']  = df_games['pgn'].str[3:]

df_games.head()

,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result,pgn
0,666000,2380.0,NaN,81,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. Nxd...
1,666001,2350.0,NaN,80,tourn,1-0,1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
2,666002,2355.0,NaN,89,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. Bg5...
3,666003,NaN,2365.0,30,tourn,1/2-1/2,1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. Bc4 ...
4,666004,NaN,2370.0,72,tourn,1-0,1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...


## Filtro de caracteres indesejados

Conjuntos com cifrão, número com 3 pontos

In [ ]:
df_games['pgn'] = df_games['pgn'].apply(lambda g: re.sub(r'\$\d ', '', g)) # remove cifrões

df_games['pgn'] = df_games['pgn'].apply(lambda g: re.sub(r' \d\d\.\.\. ', '', g)) # remove lances com 3 pontos

df_games.head()

,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result,pgn
0,666000,2380.0,NaN,81,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. Nxd...
1,666001,2350.0,NaN,80,tourn,1-0,1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...
2,666002,2355.0,NaN,89,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. Bg5...
3,666003,NaN,2365.0,30,tourn,1/2-1/2,1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. Bc4 ...
4,666004,NaN,2370.0,72,tourn,1-0,1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...


# Criação do vocabulário do xadrez

In [ ]:
linhas = ['1', '2', '3', '4', '5', '6', '7', '8']
colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

pecas = ['N', 'B', 'R', 'Q', 'K']
variacoes = ['+', '#']

casas = []

for coluna in colunas:
    for linha in linhas:
        casas.append(coluna + linha)

# Movimentos comuns

movimentos_comuns = []
movimentos_comuns_com_indicador = []

movimentos_comuns_captura = []
movimentos_comuns_captura_com_indicador = []

for peca in pecas:
    for casa in casas:
        movimentos_comuns.append(peca + casa) # movimentos comuns

        movimentos_comuns_captura.append(peca + 'x' + casa) # movimentos com captura

        if peca in ('N', 'R'):
            for indicador in linhas + colunas:
                movimentos_comuns_com_indicador.append(peca + indicador + casa)
                movimentos_comuns_captura_com_indicador.append(peca + indicador + 'x' + casa)

# Movimentos peoes

movimentos_peoes = casas.copy()
movimentos_peoes_com_captura = []
movimentos_peoes_promocao = []

for coluna in colunas:
    for casa in casas:
        movimentos_peoes_com_captura.append(coluna + 'x' + casa)

for movimento_peao in movimentos_peoes + movimentos_peoes_com_captura:
    if movimento_peao[-1] in ('1', '8'):
        for promocao in pecas:
            if promocao != 'K':
                movimentos_peoes_promocao.append(movimento_peao + '=' + promocao)

print(f'{len(movimentos_comuns)} movimentos comuns')
print(f'{len(movimentos_comuns_com_indicador)} movimentos comuns com indicador')
print(f'{len(movimentos_comuns_captura)} movimentos comuns com captura')
print(f'{len(movimentos_comuns_captura_com_indicador)} movimentos comuns com captura com indicador')
print(f'{len(movimentos_peoes)} movimentos de peões')
print(f'{len(movimentos_peoes_com_captura)} movimentos de peões com captura')
print(f'{len(movimentos_peoes_promocao)} movimentos de peões com promoção')

roques = ['O-O-O', 'O-O']

todos_movimentos = movimentos_comuns + movimentos_comuns_captura + movimentos_comuns_captura_com_indicador +\
 movimentos_comuns_com_indicador + movimentos_peoes + movimentos_peoes_com_captura + movimentos_peoes_promocao + roques

cheques = [movimento + '+' for movimento in todos_movimentos]
mates = [movimento + '#' for movimento in todos_movimentos]

todos_movimentos = todos_movimentos + cheques + mates

print(f'{len(todos_movimentos)} movimentos mapeados')

320 movimentos comuns
2048 movimentos comuns com indicador
320 movimentos comuns com captura
2048 movimentos comuns com captura com indicador
64 movimentos de peões
512 movimentos de peões com captura
576 movimentos de peões com promoção
17670 movimentos mapeados


# Tratamento do PGN para o modelo (remoção das informações extras)

In [ ]:
df_games['game_text'] = df_games['pgn'].apply(lambda g: re.sub(r'( \d+\.)|(1\. )|( \d-\d)|( 1/2-1/2)|\'', '', g))

df_games.head()

,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result,pgn,game_text
0,666000,2380.0,NaN,81,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. Nxd...,d4 Nf6 Nf3 e6 c4 c5 g3 cxd4 Nxd4 d5 Bg2 e5 Nc2...
1,666001,2350.0,NaN,80,tourn,1-0,1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
2,666002,2355.0,NaN,89,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. Bg5...,d4 Nf6 Nf3 d5 c4 e6 Nc3 Be7 Bg5 h6 Bxf6 Bxf6 e...
3,666003,NaN,2365.0,30,tourn,1/2-1/2,1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. Bc4 ...,e4 Nf6 e5 Nd5 d4 d6 Nf3 g6 Bc4 Nb6 Bb3 Bg7 Ng5...
4,666004,NaN,2370.0,72,tourn,1-0,1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...,d4 d5 c4 e6 Nc3 Be7 Nf3 Nf6 Bf4 O-O e3 c5 dxc5...


In [ ]:
import numpy as np

In [ ]:
df_games['valid_game'] = df_games['game_text'].apply(lambda x: all(np.isin(x.split(' '), todos_movimentos, assume_unique=True)))

In [ ]:
df_games['valid_game'].value_counts(normalize=True)

True     0.950808
False    0.049192
Name: valid_game, dtype: float64

In [ ]:
df_valid_games = df_games[(df_games['valid_game']) & (df_games['game_text'] != '')]

print(df_valid_games.shape)

(2375140, 9)


In [ ]:
df_valid_games.head()

,id_partida,WhiteElo,BlackElo,NbMoves,EventType,Result,pgn,game_text,valid_game
0,666000,2380.0,NaN,81,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 e6 3. c4 c5 4. g3 cxd4 5. Nxd...,d4 Nf6 Nf3 e6 c4 c5 g3 cxd4 Nxd4 d5 Bg2 e5 Nc2...,True
1,666001,2350.0,NaN,80,tourn,1-0,1. d4 Nf6 2. c4 g6 3. Nc3 Bg7 4. e4 d6 5. Nf3 ...,d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...,True
2,666002,2355.0,NaN,89,tourn,1/2-1/2,1. d4 Nf6 2. Nf3 d5 3. c4 e6 4. Nc3 Be7 5. Bg5...,d4 Nf6 Nf3 d5 c4 e6 Nc3 Be7 Bg5 h6 Bxf6 Bxf6 e...,True
3,666003,NaN,2365.0,30,tourn,1/2-1/2,1. e4 Nf6 2. e5 Nd5 3. d4 d6 4. Nf3 g6 5. Bc4 ...,e4 Nf6 e5 Nd5 d4 d6 Nf3 g6 Bc4 Nb6 Bb3 Bg7 Ng5...,True
4,666004,NaN,2370.0,72,tourn,1-0,1. d4 d5 2. c4 e6 3. Nc3 Be7 4. Nf3 Nf6 5. Bf4...,d4 d5 c4 e6 Nc3 Be7 Nf3 Nf6 Bf4 O-O e3 c5 dxc5...,True


In [ ]:
df_export = df_games[['game_text']]

In [ ]:
df_export.columns = ['']

In [ ]:
print(df_export.shape)

df_export.head()

(2498023, 1)


,
0,d4 Nf6 Nf3 e6 c4 c5 g3 cxd4 Nxd4 d5 Bg2 e5 Nc2...
1,d4 Nf6 c4 g6 Nc3 Bg7 e4 d6 Nf3 O-O Be2 e5 O-O ...
2,d4 Nf6 Nf3 d5 c4 e6 Nc3 Be7 Bg5 h6 Bxf6 Bxf6 e...
3,e4 Nf6 e5 Nd5 d4 d6 Nf3 g6 Bc4 Nb6 Bb3 Bg7 Ng5...
4,d4 d5 c4 e6 Nc3 Be7 Nf3 Nf6 Bf4 O-O e3 c5 dxc5...


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_export_train, df_export_valid = train_test_split(df_export, test_size=.1)

In [ ]:
df_export_train.to_csv(f'{PATH_TO_SAVE_FILES}/games_train.txt', index=False)

In [ ]:
df_export_valid.to_csv(f'{PATH_TO_SAVE_FILES}/games_valid.txt', index=False)